## ViT train & infer

In [ ]:
!pip install transformers
!pip install accelerate

In [ ]:
%time
!python train_CLIP_ViT_pairwise.py --model_name 'openai/clip-vit-large-patch14-336' --train_data_file 'data/train_vit_pairwise.json' --test_data_file 'data/test_vit_pairwise.json' \
--image_dir 'data/imgs' --max_epoch 1

In [ ]:
%time
!python infer_Clip_ViT_pairwise.py  --model_name 'vit_pairwise_clip_model/model.pth' --test_data_file 'data/test_vit.json' --image_dir 'data/imgs' --save_path 'vit_pairwise_clip_model'

In [ ]:
%time
!python train_Clip_ViT_pairwise_emb.py --model_name 'openai/clip-vit-large-patch14-336' --train_data_file 'data/train_vit_pairwise_text.json' --test_data_file 'data/test_vit_pairwise_text.json' \
--image_dir 'data/imgs' --max_epoch 1

In [ ]:
%time
!python infer_Clip_ViT_pairwise_emb.py  --model_name 'vit_pairwise_model_emb/model.pth' --test_data_file 'data/test_vit_text.json' --image_dir 'data/imgs' 

In [ ]:
print(213.89/287*10000)
213.89/287*10000*2000/60/60*0.736

## eval

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
pred=pd.read_csv('vit_pairwise_clip_model/pred.csv')
pred.head()
pp=[1 if i>0.5 else 0 for i in pred['predict'].values]
ll=[int(i) for i in pred['label'].values]
print(classification_report(ll, pp))

In [ ]:
!python eval.py --data_path 'vit_pairwise_clip_model/pred_single_sample.csv'

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
pred=pd.read_csv('vit_pairwise_model_emb/pred.csv')
pred.head()
pp=[1 if i>0.5 else 0 for i in pred['predict'].values]
ll=[int(i) for i in pred['label'].values]
print(classification_report(ll, pp))

In [ ]:
!python eval.py --data_path 'vit_pairwise_model_emb/pred_single_sample.csv'

## badcase analysis

In [ ]:
import pandas as pd

pred=pd.read_csv('./vit_pairwise_clip_model/pred_single_sample.csv')
pred['hotel_id']=[i.split('_')[0] for i in pred['image'].values]
pred.sort_values('predict',ascending=False,inplace=True)

dd=[]
grouped=pred.groupby('hotel_id')
for _,group in grouped:
    group['pred_label']=[1]+[0]*(group.shape[0]-1)
    dd.append(group)
dd=pd.concat(dd)

from sklearn.metrics import classification_report
print(classification_report(dd['label'].values, dd['pred_label'].values))


dw=dd[dd['label']!=dd['pred_label']]
ids=dw['hotel_id'].unique()
from PIL import Image
dw.head()

from PIL import Image

def resize_image(input_image_path,resize=512):
    original_image = Image.open(input_image_path)
    width, height = original_image.size

    if width >= height:
        new_width = resize
        new_height = int((resize / width) * height)
    else:
        new_height = resize
        new_width = int((resize / height) * width)

    resized_image = original_image.resize((new_width, new_height))

    return resized_image

from PIL import Image, ImageDraw, ImageFont

def combine_images(image1, image2,  title1, title2):
    # 加载两张图片

    # 创建新的图像，大小为两张图片宽度之和，高度取两张图片中高度较大的那个
    new_width = image1.width + image2.width
    new_height = max(image1.height, image2.height)
    new_image = Image.new('RGB', (new_width, new_height))

    # 在新图像上绘制两张图片
    new_image.paste(image1, (0, 0))
    new_image.paste(image2, (image1.width, 0))

    # 添加标题
    draw = ImageDraw.Draw(new_image)
    font = ImageFont.load_default()
    draw.text((10, 10), title1, fill="black", font=font)
    draw.text((image1.width + 10, 10), title2, fill="black", font=font)

    return new_image
    
for id in ids:
    dwi=dw[dw['hotel_id']==id]
    print(dwi)
    for img,predict,label,hotel_id,pred in dwi.values:
        if label==1:
            label1=img
        elif pred==1:
            pred1=img
    print('hotel_id:',id)
    # print('label pic:')
    img1=resize_image('data/imgs/'+label1)
    # print('predict pic:') 
    img2=resize_image('data/imgs/'+pred1)
    combine_images(img1, img2,  'groud truth', 'predict').show()
    # print('-----------------------')
    
   

In [ ]:

dr=dd[dd['label']==dd['pred_label']]
ids=dr['hotel_id'].sample(10).unique()
for id in ids:
    dwi=dr[dr['hotel_id']==id]
    if sum(dwi['label'].values)==0:
        continue
    print(dwi)
    for img,predict,label,hotel_id,pred in dwi.values:
        if label==1:
            label1=img
        if pred==1:
            pred1=img
    print('hotel_id:',id)
    # print('label pic:')
    img1=resize_image('../data/imgs/'+label1).show()

    